In [1]:
# -*- coding: utf-8 -*-

"""
[Assignment 5] Evaluation Notebook
- 저장된 모델 로드
- Test Dataset 로드
- ROUGE Score 측정 및 결과 분석
"""

# ==========================================
# 1. 환경 설정 및 데이터 로드
# ==========================================
import sys
import subprocess

try:
    import evaluate
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "evaluate", "rouge_score", "transformers", "datasets", "accelerate"])

from google.colab import drive
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
import evaluate
from tqdm import tqdm

# Google Drive 마운트
drive.mount('/content/drive')

# 경로 설정 (Training에서 저장한 경로와 동일해야 함)
PROJECT_DIR = "/content/drive/MyDrive/NLP_Assignment5"
MODEL_SAVE_DIR = os.path.join(PROJECT_DIR, "final_model")
DATA_SAVE_DIR = os.path.join(PROJECT_DIR, "data")

# Test 데이터 로드
test_csv_path = os.path.join(DATA_SAVE_DIR, "test_dataset.csv")
if not os.path.exists(test_csv_path):
    raise FileNotFoundError("Test Dataset을 찾을 수 없습니다. Training을 먼저 실행하세요.")

test_df = pd.read_csv(test_csv_path)
print(f"📂 Test Data 로드 완료: {len(test_df)}건")


# ==========================================
# 2. 모델 및 토크나이저 로드
# ==========================================
print(f"🤖 저장된 모델 로드 중... ({MODEL_SAVE_DIR})")
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_SAVE_DIR)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_SAVE_DIR).to(device)


# ==========================================
# 3. 평가 지표 설정 (ROUGE)
# ==========================================
rouge = evaluate.load('rouge')

def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            max_length=128,
            min_length=30,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


# ==========================================
# 4. 성능 평가 (Inference Loop)
# ==========================================
print("\n🧪 평가 진행 중 (ROUGE Score 계산)...")

predictions = []
references = []

# 진행상황 표시를 위해 tqdm 사용
for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
    doc = row['document']
    ref = row['summary']

    pred = generate_summary(doc)

    predictions.append(pred)
    references.append(ref)

# ROUGE 점수 계산
results = rouge.compute(predictions=predictions, references=references)

# ==========================================
# 5. 결과 출력 및 저장
# ==========================================
print("\n📊 [Final Evaluation Results]")
print("-" * 30)
print(f"ROUGE-1: {results['rouge1']:.4f}")
print(f"ROUGE-2: {results['rouge2']:.4f}")
print(f"ROUGE-L: {results['rougeL']:.4f}")
print("-" * 30)

# 결과 DataFrame 저장
eval_result_df = pd.DataFrame({
    'document': test_df['document'],
    'reference': references,
    'prediction': predictions
})
eval_result_df.to_csv("evaluation_results.csv", index=False, encoding='utf-8-sig')
print("✅ 상세 결과가 'evaluation_results.csv'에 저장되었습니다.")

# 샘플 출력
print("\n👀 [샘플 결과 확인]")
for i in range(3):
    print(f"\nExample {i+1}:")
    print(f"Original : {test_df.iloc[i]['document'][:100]}...")
    print(f"Reference: {references[i]}")
    print(f"Generated: {predictions[i]}")

Mounted at /content/drive
📂 Test Data 로드 완료: 2046건
🤖 저장된 모델 로드 중... (/content/drive/MyDrive/NLP_Assignment5/final_model)


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



🧪 평가 진행 중 (ROUGE Score 계산)...


100%|██████████| 2046/2046 [35:41<00:00,  1.05s/it]



📊 [Final Evaluation Results]
------------------------------
ROUGE-1: 0.5849
ROUGE-2: 0.3771
ROUGE-L: 0.5539
------------------------------
✅ 상세 결과가 'evaluation_results.csv'에 저장되었습니다.

👀 [샘플 결과 확인]

Example 1:
Original : IPTV 3사 손잡고 콘텐츠 확보 글로벌 OTT와 본격 경쟁 예고 서울경제 KT 030200 ·SK브로드밴드· LG유플러스 032640 등 인터넷TV IPTV 3사가 3000억 원...
Reference: KTV IPTV 3사가 3000억 원을 공동 투자해 국내 미디어 생태계 보호 및 IPTV 플랫폼 가치 강화를 목적으로 ‘콘텐츠 공동 전략 수급을 위한 업무협약’을 체결하고 공동 브랜드 구축과 콘텐츠 공동 프로모션 등 적극적인 사업협력도 검토한다고 밝혔다.
Generated: 8일 KT·SK브로드밴드·LG유플러스는 국내 미디어 생태계 보호 및 IPTV 플랫폼 가치 강화를 목적으로 ‘콘텐츠 공동 전략 수급을 위한 업무협약’을 체결하고 3000억 원 규모의 투자를 진행해 한국영화 및 드라마 제작사 투자사 등과 협업을 확대해 나갈 계획이라고 밝혔다. IPTV 3사는 공동수급 운영위원회를 통해 3000억 원 규모의 투자를 진행할 예정이다. 이를 통해 한국영화 및 드라마 제작사 투자사 등과 협업을 확대해 나갈 계획이다. 3사가 협업에 나서게 된 배경에는 글로벌 온라인동영상서비스 OTT 의 공격적인 국내 진출이 있다. 3사 관계자는 “특정 콘텐츠에 대한 단일 플랫폼

Example 2:
Original : 뉴스투데이 앵커 연결선 없이 귀에 꽂고 다니는 무선 이어폰 시장 규모가 올해 6억 원 대를 돌파할 것이란 전망 전해드린 적 있는데요. 단순한 이어폰이 아니라 건강관리나 장애 보조 ...
Reference: 휴선 없이 귀에 꽂고 다니는 무선 이어폰 시장 규모가 올해 6억 원 대를 돌